In [1]:
from __future__ import print_function
import torch
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
from torchvision import datasets, transforms

In [2]:
batch_size = 64

In [3]:
train_datset = datasets.MNIST(root='./mnist_data',
                             train=True,
                             transform=transforms.ToTensor())

In [6]:
test_dataset = datasets.MNIST(root='./mnist_data',
                             train=False,
                             transform=transforms.ToTensor())

In [7]:
train_loader = torch.utils.data.DataLoader(dataset=train_datset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [8]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

In [9]:
class SimpleInception(torch.nn.Module):
    
    def __init__(self, in_channels):
        super(SimpleInception, self).__init__()
        self.branch1x1 = torch.nn.Conv2d(in_channels, 16, kernel_size=1)
        
        self.branch5x5_1 = torch.nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = torch.nn.Conv2d(16, 24, kernel_size=5, padding=2)
        
        self.branch3x3dbl_1 = torch.nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = torch.nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = torch.nn.Conv2d(24, 24, kernel_size=3, padding=1)
        
        self.branch_pool = torch.nn.Conv2d(in_channels, 24, kernel_size=1)
        
    def forward(self, x):
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)
        
        branch_pool = torch.nn.functional.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)
        
        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

In [19]:
class NNet(torch.nn.Module):
    
    def __init__(self):
        super(NNet, self).__init__()
        
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(88, 20, kernel_size=5)
        
        self.inception1 = SimpleInception(in_channels=10)
        self.inception2 = SimpleInception(in_channels=20)
        
        self.mp = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(1408, 10)
        
    def forward(self, x):
        in_size = x.size(0)
        x = torch.nn.functional.relu(self.mp(self.conv1(x)))
        x = self.inception1(x)
        x = torch.nn.functional.relu(self.mp(self.conv2(x)))
        x = self.inception2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return torch.nn.functional.log_softmax(x)

In [20]:
model = NNet()

In [21]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [25]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = torch.nn.functional.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [26]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += torch.nn.functional.nll_loss(output, target, size_average=False).data[0]
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309072
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.287190
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.116315
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.680324
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.368239
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.504674
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.389219
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.283463
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.325551
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.230101

Test set: Average loss: 0.1871, Accuracy: 9411/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.086404
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.111793
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.144683
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.183078
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.255718
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.220983
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.057798
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.104173
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.118954
T